In [1]:
import json, pandas as pd
from tqdm import tqdm
from md_detector import MetalinguisticDisagreementDetector

In [2]:
trex_runs = {
    "gpt-3.5-turbo": json.load(open('experiments5/gpt-3.5-turbo-trex.json', 'r')),
    "gpt-4o-2024-05-13": json.load(open('experiments5/gpt-4o-2024-05-13-trex.json', 'r')),
    "gpt-4-0125-preview": json.load(open('experiments5/gpt-4-0125-preview-trex.json', 'r')),
    "mistralai/Mistral-7B-Instruct-v0.3": json.load(open('experiments5/Mistral-7B-Instruct-v0.3-trex.json', 'r')),
    "mistralai/Mixtral-8x7B-Instruct-v0.1": json.load(open('experiments5/Mixtral-8x7B-Instruct-v0.1-trex.json', 'r')),
    "meta-llama/Meta-Llama-3-70b-Instruct": json.load(open('experiments5/Meta-Llama-3-70B-Instruct-trex.json', 'r')),
    "claude-3-opus-20240229": json.load(open('experiments5/claude-3-opus-20240229-trex.json', 'r')),
    "claude-3-5-sonnet-20240620": json.load(open('experiments5/claude-3-5-sonnet-20240620-trex.json', 'r')),
    "claude-3-haiku-20240307": json.load(open('experiments5/claude-3-haiku-20240307-trex.json', 'r')),
}

md_detector = MetalinguisticDisagreementDetector(model="gpt-4o-2024-05-13")

/Users/bradleyallen/.pyenv/versions/3.11.4/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [3]:
lmds = []
for model in tqdm(trex_runs):
    results = trex_runs[model]
    fns = [ result for result in results if result["predicted"] == "0" ]
    md_results = md_detector.chain.batch(fns)
    mds_detected = [ result for result in md_results if result["text"]["answer"] == "1"]
    for md in mds_detected:
        md['md_rationale'] = md['text']['md_rationale']
        md.pop('text')
        md['model'] = model
    lmds += mds_detected

100%|██████████| 9/9 [04:59<00:00, 33.33s/it] 


In [4]:
len(lmds)

209

In [6]:
df = pd.DataFrame.from_records(lmds)

In [7]:
len(df)

209

In [8]:
df.groupby('model').size()

model
claude-3-5-sonnet-20240620              20
claude-3-haiku-20240307                  8
claude-3-opus-20240229                  15
gpt-3.5-turbo                           25
gpt-4-0125-preview                      15
gpt-4o-2024-05-13                        7
meta-llama/Meta-Llama-3-70b-Instruct    25
mistralai/Mistral-7B-Instruct-v0.3      63
mistralai/Mixtral-8x7B-Instruct-v0.1    31
dtype: int64

In [11]:
fn_df = df[(df['model'] == 'claude-3-haiku-20240307')].sort_values("s_label")

In [12]:
fns_styler = fn_df.style.set_properties(**{"text-align": "left", "vertical-align" : "top", "overflow-wrap": "break-word"}).hide(axis="index")
display(fns_styler)

s_id,s_label,p_id,p_label,p_definition,o_id,o_label,world,rationale,predicted,md_rationale,model
Q207353,Acadia,P17,country,sovereign state that this item is in (not to be used for human beings),Q16,Canada,"Cajun cuisine (French: Cuisine cadienne, [kɥizin kadjɛn]) is a style of cooking named for the French-speaking Acadian people deported by the British from Acadia in Canada to the Acadiana region of Louisiana. It is what could be called a rustic cuisine; locally available ingredients predominate and preparation is simple. An authentic Cajun meal is usually a three-pot affair, with one pot dedicated to the main dish, one dedicated to steamed rice, special made sausages, or some seafood dish, and the third containing whatever vegetable is plentiful or available. Shrimp and pork sausage are staple meats used in a variety of dishes. The aromatic vegetables green bell pepper (poivron), onion, and celery are called the holy trinity by Cajun chefs in Cajun and Creole cuisines. Roughly diced and combined in cooking, the method is similar to the use of the mirepoix in traditional French cuisine which blends roughly diced onion, celery and carrot. Characteristic aromatics for the Creole version may also include parsley, bay leaf, green onions, dried cayenne pepper, and dried black pepper.","The given knowledge graph triple states that the subject ""Acadia"" is in the country (sovereign state) of ""Canada"". However, based on the information provided in the hypothetical world, Acadia refers to the region from which the Acadian people were deported by the British to the Acadiana region of Louisiana. The text does not mention Acadia being located in Canada, but rather that the Acadians were originally from Acadia in Canada. Therefore, the statement that Acadia is in the country of Canada is false in the given hypothetical world.",0,"To determine if the rationale provided is an example of a metalinguistic disagreement, we need to understand what a metalinguistic disagreement entails. A metalinguistic disagreement occurs when the parties involved are not directly disagreeing about the facts of the matter, but rather about the use of language or the definitions of terms. In this case, the disagreement seems to be about the interpretation of the term ""Acadia"" and its geographical and historical context. The given knowledge graph triple states that ""Acadia"" is in ""Canada,"" which is factually correct based on historical and geographical knowledge. Acadia was a region in what is now eastern Canada, primarily in the Maritime provinces. The rationale provided argues that the statement is false because it interprets the text to mean that Acadia is not currently in Canada, but rather that the Acadian people were deported from Acadia to Louisiana. This interpretation seems to be focusing on the historical movement of people rather than the current geographical location of Acadia. Therefore, the disagreement here is not about the factual location of Acadia but rather about how the term ""Acadia"" is being used and interpreted in the context of the historical narrative provided. This makes it a metalinguistic disagreement. ###",claude-3-haiku-20240307
Q1405,Augustus,P2359,Roman nomen gentilicium,"standard part of the name of a Roman, link to items about the Roman gens only",Q1759265,Octavia gens,"Claudius (/ˈklɔːdiəs/; Latin: Tiberius Claudius Caesar Augustus Germanicus; 1 August 10 BC – 13 October 54 AD) was Roman emperor from 41 to 54. A member of the Julio-Claudian dynasty, he was the son of Drusus and Antonia Minor. He was born at Lugdunum in Gaul, the first Roman Emperor to be born outside Italy. Because he was afflicted with a limp and slight deafness due to sickness at a young age, his family ostracized him and excluded him from public office until his consulship, shared with his nephew Caligula in 37. Claudius' infirmity probably saved him from the fate of many other nobles during the purges of Tiberius and Caligula's reigns; potential en